In [ ]:
#import relevant libraries

import os
import sys
#only need to run these if packages haven't been installed yet
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install datetime
#!{sys.executable} -m pip install seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read and clean up ABCD data
# set base dirctories
ABCD_base_dir   = 'base_directory_path'

#load subj fc data
ABCD_fc_df = pd.read_csv(os.path.join(ABCD_base_dir, 'fc_data.csv'), header=None)
ABCD_fc_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'fc_subj_data.txt'), header=None)
ABCD_fc = ABCD_fc_df.T

# load subj demo and clinical data
ABCD_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'clin_subj_data.csv'))
ABCD_clin = pd.read_csv(os.path.join(ABCD_base_dir, 'clin_subj_data.csv'))

#drop duplicate header rows
header_row = 0
ABCD_subj = ABCD_subj.drop(header_row)
ABCD_clin = ABCD_clin.drop(header_row)


In [ ]:
#add subj key data to fc data and sort
ABCD_fc_subj.columns=['subjectkey']
ABCD_fc.insert(0, "subjectkey", ABCD_fc_subj, True)
ABCD_fc_sorted = ABCD_fc.sort_values(by='subjectkey', ascending=True)


In [ ]:
#clean and sort clinical data
mask = ABCD_clin.subjectkey.isin(ABCD_fc_sorted['subjectkey'])
ABCD_clin_subjs = ABCD_clin[mask]
ABCD_clin_baseline = ABCD_clin_subjs[ABCD_clin_subjs.eventname == 'baseline_year_1_arm_1']
ABCD_clin_sorted = ABCD_clin_baseline.sort_values(by='subjectkey', ascending=True)

In [ ]:
#clean and sort subject data
mask = ABCD_subj.subjectkey.isin(ABCD_fc_sorted['subjectkey'])
ABCD_subj_incl = ABCD_subj[mask]
ABCD_subj_baseline = ABCD_subj_incl[ABCD_subj_incl.eventname == 'baseline_year_1_arm_1']
ABCD_subj_sorted = ABCD_subj_baseline.sort_values(by='subjectkey', ascending=True)
ABCD_subj_data = ABCD_subj_sorted

In [ ]:
#isolate and clean clinical variables to be predicted
ABCD_clin_data = ABCD_clin_sorted[['cbcl_scr_syn_anxdep_r', 'cbcl_scr_syn_withdep_r',
                                  'cbcl_scr_syn_somatic_r', 'cbcl_scr_syn_social_r',
                                  'cbcl_scr_syn_thought_r', 'cbcl_scr_syn_attention_r',
                                  'cbcl_scr_syn_rulebreak_r', 'cbcl_scr_syn_aggressive_r',
                                  'cbcl_scr_syn_internal_r',  'cbcl_scr_syn_external_r',
                                  'cbcl_scr_syn_totprob_r', 'cbcl_scr_dsm5_depress_r',
                                  'cbcl_scr_dsm5_anxdisord_r', 'cbcl_scr_dsm5_somaticpr_r',
                                  'cbcl_scr_dsm5_adhd_r', 'cbcl_scr_dsm5_opposit_r',
                                  'cbcl_scr_dsm5_conduct_r', 'cbcl_scr_07_sct_r',
                                  'cbcl_scr_07_ocd_r', 'cbcl_scr_07_stress_r']]

ABCD_clin_labels = ['AnxDep', 'WithDep', 'Somatic', 'Social', 'Thought', 'Attention',
                    'RuleBreak', 'Aggresive', 'Internal', 'External', 'TotProb', 'Depress',
                    'AnxDiscord', 'SomaticPr', 'ADHD', 'Opposit', 'Conduct', 'Sluggish', 
                    'OCD', 'Stress']

ABCD_clin_data.columns = ABCD_clin_labels
ABCD_clin_data.reset_index(inplace=True)
ABCD_clin_data = ABCD_clin_data.drop(columns=['index'])

#clean fc data 
ABCD_fc_data = ABCD_fc_sorted.drop(columns=['subjectkey'])
ABCD_fc_data.reset_index(inplace=True) 
ABCD_fc_data = ABCD_fc_data.drop(columns=['index'])

#clean subj data
ABCD_subj_data.reset_index(inplace=True) 
ABCD_subj_data = ABCD_subj_data.drop(columns=['index'])



In [ ]:
#get sex-specific variables
mask_m = ABCD_subj_sorted.sex=='M'
ABCD_subj_m = ABCD_subj_sorted[mask_m]
ABCD_clin_m = ABCD_clin_data[mask_m]
ABCD_fc_m = ABCD_fc_data[mask_m]

mask_f = ABCD_subj_sorted.sex=='F'
ABCD_subj_f = ABCD_subj_sorted[mask_f]
ABCD_clin_f = ABCD_clin_data[mask_f]
ABCD_fc_f = ABCD_fc_data[mask_f]

In [ ]:
#number of repetitions you want to perform
rep = 100
#number of folds you want in the cross-validation
k = 3
#proportion of data you want in your training set and test set
train_size = .66
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')

#set hyperparameter grid space you want to search through for the model
#adapted from the Thomas Yeo Lab Github: 
#ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
#alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
#          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 2000, 
#          3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

alphas = [0.001, 0.01, 0.1, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000]


#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}



#set x data to be the input variable you want to use
X_m = ABCD_fc_m
X_f = ABCD_fc_f


#set y data to be clin var you want to use
#do one at a time to prevent crashes
clin_var = 'AnxDep'
Y_m = ABCD_clin_m[clin_var]
Y_f = ABCD_clin_f[clin_var]

#ABCD_clin_labels = ['AnxDep', 'WithDep', 'Somatic', 'Social', 'Thought', 'Attention',
#                    'RuleBreak', 'Aggresive', 'Internal', 'External', 'TotProb', 'Depress',
#                    'AnxDiscord', 'SomaticPr', 'ADHD', 'Opposit', 'Conduct', 'Sluggish', 
#                    'OCD', 'Stress']



#number of variables 
#is 1 since training models to predict one clin var at a time
n_beh = 1

#number of features 
n_feat = X_f.shape[1]

#test within sex only here
n_test = 1

In [ ]:
#create arrays to store variables

#r^2 - coefficient of determination
r2_m = np.zeros([rep])
r2_f = np.zeros([rep])
#explained variance
var_m = np.zeros([rep])
var_f = np.zeros([rep])
#correlation between true and predicted (aka prediction accuracy)
corr_m = np.zeros([rep])
corr_f = np.zeros([rep])
#optimised alpha (hyperparameter)
opt_alpha_m = np.zeros([rep])
opt_alpha_f = np.zeros([rep])
#feature importance extracted from the model
featimp_m = np.zeros([rep,n_feat])
featimp_f = np.zeros([rep,n_feat])
#for when the feat weights get haufe-inverted
#featimp_haufe_m = np.zeros([rep,n_feat])
#featimp_haufe_f = np.zeros([rep,n_feat])

In [ ]:
#iterate through number of models
for p in range(rep):
    
    #print model # you're on
    print('Model %d' %(p+1))
    
    #print time
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    
    #split into train/test data
    train_inds_m, test_inds_m = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X_m, groups=ABCD_subj_m['site_id_l']))
    train_inds_f, test_inds_f = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X_f, groups=ABCD_subj_f['site_id_l']))
    #x_train, x_test, cog_train, cog_test = train_test_split(X, Y, test_size=1-train_size, shuffle=True, random_state=p)
    
    #set x values based on indices from split
    x_train_m = X_m.iloc[train_inds_m].values
    x_test_m = X_m.iloc[test_inds_m].values
        
    #set y values based on indices from split  
    beh_train_m = Y_m.iloc[train_inds_m].values
    beh_test_m = Y_m.iloc[test_inds_m].values 
    
    #set x values based on indices from split
    x_train_f = X_f.iloc[train_inds_f].values
    x_test_f = X_f.iloc[test_inds_f].values
        
    #set y values based on indices from split  
    beh_train_f = Y_f.iloc[train_inds_f].values
    beh_test_f = Y_f.iloc[test_inds_f].values 
    
    #convert y values to to double
    y_train_m = np.double(beh_train_m)
    y_test_m = np.double(beh_test_m)
        
    y_train_f = np.double(beh_train_f)
    y_test_f = np.double(beh_test_f)


    #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
    best_scores_m = []
    best_params_m = []
    
    best_scores_f = []
    best_params_f = []
        
        
    #set parameters for inner and outer loops for CV
    cv_split = GroupKFold(n_splits=k)
        
    print ("Optimising Male-Trained Models")
            
    #define regressor with grid-search CV for inner loop
    gridSearch_m = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')

    #fit regressor to the model, use site ID as group category again
    gridSearch_m.fit(x_train_m, y_train_m, groups=ABCD_subj_m.iloc[train_inds_m]['site_id_l'])

    #save parameters corresponding to the best score
    best_params_m.append(list(gridSearch_m.best_params_.values()))
    best_scores_m.append(gridSearch_m.best_score_)
        
    print ("Optimising Female-Trained Models")
        
    #define regressor with grid-search CV for inner loop
    gridSearch_f = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')

    #fit regressor to the model, use site ID as group category again
    gridSearch_f.fit(x_train_f, y_train_f, groups=ABCD_subj_f.iloc[train_inds_f]['site_id_l'])

    #save parameters corresponding to the best score
    best_params_f.append(list(gridSearch_f.best_params_.values()))
    best_scores_f.append(gridSearch_f.best_score_)
        
        
    print ("Evaluating Models")
        
    #save optimised alpha values
    opt_alpha_m[p] = best_params_m[best_scores_m.index(np.max(best_scores_m))][0]
    opt_alpha_f[p] = best_params_f[best_scores_f.index(np.max(best_scores_f))][0]

    #fit optimized models
    model_m = Ridge(alpha = opt_alpha_m[p], normalize=True, max_iter=1000000, solver='lsqr')
    model_m.fit(x_train_m, y_train_m);
    
    model_f = Ridge(alpha = opt_alpha_f[p], normalize=True, max_iter=1000000, solver='lsqr')
    model_f.fit(x_train_f, y_train_f);
        
        
    #evaluate model within sex within behavior
    r2_m[p]=model_m.score(x_test_m,y_test_m)
    r2_f[p]=model_f.score(x_test_f,y_test_f)
        

    #predict within sex
    preds_m = []
    preds_f = []
        
    preds_m = model_m.predict(x_test_m).ravel()
    preds_f = model_f.predict(x_test_f).ravel()
        
        
    #compute explained variance 
    var_m[p] = explained_variance_score(y_test_m, preds_m)
    var_f[p] = explained_variance_score(y_test_f, preds_f)


    #compute correlation between true and predicted (prediction accuracy)
    corr_m[p] = np.corrcoef(y_test_m.ravel(), preds_m)[1,0]
    corr_f[p] = np.corrcoef(y_test_f.ravel(), preds_f)[1,0]

    
    #print ("Haufe-Transforming Feature Weights")
    #cov_x = []
    #cov_y = []
    
    #extract feature importance
    #featimp_m[p,:] = model_m.coef_
    #featimp_f[p,:] = model_f.coef_
    #compute Haufe-inverted feature weights
    #cov_x_m = np.cov(np.transpose(x_train_m))
    #cov_y_m = np.cov(y_train_m)
    #featimp_haufe_m[p,:] = np.matmul(cov_x_m,featimp_m[p,:])*(1/cov_y_m)
        
    #cov_x_f = np.cov(np.transpose(x_train_f))
    #cov_y_f = np.cov(y_train_f)
    #featimp_haufe_f[p,:] = np.matmul(cov_x_f,featimp_f[p,:])*(1/cov_y_f)
        
        
    # save results
    results_dir   = 'results_directory_path'

    np.save((results_dir + '/fc_r2_m_' + clin_var + '.npy'),r2_m)
    np.save((results_dir + '/fc_var_m_' + clin_var + '.npy'),var_m)
    np.save((results_dir + '/fc_corr_m_' + clin_var + '.npy'),corr_m)
    np.save((results_dir + '/fc_alpha_m_' + clin_var + '.npy'),opt_alpha_m)
    np.save((results_dir + '/fc_featimp_m_' + clin_var + '.npy'),featimp_m)
    #np.save((results_dir + '/fc_featimp_haufe_m_' + clin_var + '.npy'),featimp_haufe_m)

    np.save((results_dir + '/fc_r2_f_' + clin_var + '.npy'),r2_f)
    np.save((results_dir + '/fc_var_f_' + clin_var + '.npy'),var_f)
    np.save((results_dir + '/fc_corr_f_' + clin_var + '.npy'),corr_f)
    np.save((results_dir + '/fc_alpha_f_' + clin_var + '.npy'),opt_alpha_f)
    np.save((results_dir + '/fc_featimp_f_' + clin_var + '.npy'),featimp_f)
    #np.save((results_dir + '/fc_featimp_haufe_f_' + clin_var + '.npy'),featimp_haufe_f)
    
    
    
        
        